In [67]:
import numpy as np
import pandas as pd
from definitions import *
from tqdm import tqdm
from scipy import stats

In [49]:
users_aggregated_data = pd.read_csv("./aggregated_data.csv", index_col=0)

In [50]:
df_pvalues = pd.DataFrame()

In [51]:
df_rvalues = pd.DataFrame()

In [52]:
data_pvalue = [["test_col",[]]]
data_rvalue = [["test_col",[]]]

for col in users_aggregated_data:
    data_pvalue.append([col,[]])
    data_rvalue.append([col,[]])
    

## Selecionando o tipo de teste que sera feito

In [78]:
target_correlation = "spearmanr"

In [54]:
def calculate_correlation(corr_function ="pearsonr"):
    for col1 in users_aggregated_data:
        data_pvalue[0][1].append(col1)
        data_rvalue[0][1].append(col1)
        for col2 in users_aggregated_data:
            if(corr_function == "pearsonr"):
                r,p = stats.pearsonr(users_aggregated_data[col1], users_aggregated_data[col2])
            if(corr_function == "spearmanr"):
                r,p = stats.spearmanr(users_aggregated_data[col1], users_aggregated_data[col2])

            for i in range(len(data_pvalue)):
                if(data_pvalue[i][0] == col2):
                    data_pvalue[i][1].append(p)
                if(data_rvalue[i][0] == col2):
                    data_rvalue[i][1].append(r)

calculate_correlation(target_correlation)

In [55]:
for dt in data_pvalue:
    df_pvalues[dt[0]] = np.array(dt[1])
for dt in data_rvalue:
    df_rvalues[dt[0]] = np.array(dt[1])

In [56]:
df_pvalues.to_csv(target_correlation+"/pvalue_tests.csv")

In [57]:
df_rvalues.to_csv(target_correlation+"/rvalue_tests.csv")

### Pvalue Filters
###### threshold = 0.005 (sobs falou)

In [79]:
df_pvalues = pd.read_csv(target_correlation+"/pvalue_tests.csv",index_col=0)

In [70]:
def run_p_filters(threshold):
    pvalue_threshold = threshold
    ref = []
    dest = []
    val = []
    for index, row in df_pvalues.iterrows():
        for col in df_pvalues:
            if(col!="test_col"):
                if(row[col]<=pvalue_threshold):
                    if(row["test_col"] in dest and col in ref):
                        continue
                    ref.append(row["test_col"])
                    dest.append(col)
                    val.append(row[col])
    signif_values_df = pd.DataFrame()
    signif_values_df["col_a"] = np.array(ref)
    signif_values_df["col_b"] = np.array(dest)
    signif_values_df["pvalue"] = np.array(val)
    signif_values_df.to_csv(target_correlation+"/pvalue_filtered/significant_pvalues-"+str(pvalue_threshold)+".csv")


In [80]:
ths_values = [0.05,0.01,0.005,0.001,0.0005,0.0001,0.000005,0.000000000000005]
for t_val in ths_values:
    run_p_filters(t_val)

In [72]:
def col_map(col):
    try:
        return int(col.split("_")[1])
    except:
        return col
def map_and_sort_results_p(threshold):
    sig_df = pd.read_csv(target_correlation+"/pvalue_filtered/significant_pvalues-"+str(threshold)+".csv",index_col = 0)
    sig_df = sig_df.sort_values(by = ['col_a', 'col_b'], ascending = [True, False], na_position = 'first')
    sig_df = sig_df[sig_df["col_b"].str.contains("score")]
    sig_df.to_csv(target_correlation+"/pvalue_filtered/significant_pvalues-"+str(threshold)+"-sorted.csv")
    try:
        sig_df["col_a"] = sig_df["col_a"].map(col_map)
    except:
        None
    sig_df = sig_df.groupby(['col_a', 'col_b']).count()
    sig_df.to_csv(target_correlation+"/pvalue_filtered/significant_pvalues-"+str(threshold)+"-count.csv")


In [81]:
for t_val in ths_values:
    map_and_sort_results_p(t_val)

### Rvalue filters

In [82]:
df_rvalues = pd.read_csv(target_correlation+"/rvalue_tests.csv",index_col=0)

In [75]:
def run_r_filters(threshold):
    rvalue_threshold = threshold
    ref = []
    dest = []
    val = []
    for index, row in df_rvalues.iterrows():
        for col in df_rvalues:
            if(col!="test_col"):
                if(row[col]<=-1*rvalue_threshold or row[col]>=rvalue_threshold):
                    if(row["test_col"] in dest and col in ref):
                        continue
                    ref.append(row["test_col"])
                    dest.append(col)
                    val.append(row[col])
    signif_values_df = pd.DataFrame()
    signif_values_df["col_a"] = np.array(ref)
    signif_values_df["col_b"] = np.array(dest)
    signif_values_df["rvalue"] = np.array(val)
    signif_values_df.to_csv(target_correlation+"/rvalue_filtered/significant_rvalues-"+str(rvalue_threshold)+".csv")

In [83]:
ths_rvalues = [0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for t_val in ths_rvalues:
    run_r_filters(t_val)

In [77]:
def map_and_sort_results_r(threshold):
    sig_df = pd.read_csv(target_correlation+"/rvalue_filtered/significant_rvalues-"+str(threshold)+".csv",index_col = 0)
    sig_df = sig_df.sort_values(by = ['col_a', 'col_b'], ascending = [True, False], na_position = 'first')
    sig_df = sig_df[sig_df["col_b"].str.contains("score")]
    sig_df.to_csv(target_correlation+"/rvalue_filtered/significant_rvalues-"+str(threshold)+"-sorted.csv")
    try:
        sig_df["col_a"] = sig_df["col_a"].map(col_map)
    except:
        None
    sig_df = sig_df.groupby(['col_a', 'col_b']).count()
    sig_df.to_csv(target_correlation+"/rvalue_filtered/significant_rvalues-"+str(threshold)+"-count.csv")

In [84]:
for t_val in ths_rvalues:
    map_and_sort_results_r(t_val)